In [1]:
import math
import numpy as np
from numba import cuda
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy.signal as signal

In [2]:
@cuda.jit
def calculate_drift(m):
    i, j = cuda.grid(2)
    if m[i,j] > .1:
        divisor = 5
        if i == 0 or m[i-1,j] > m[i,j]:
            divisor -= 1
        if j == 0 or m[i,j-1] > m[i,j]:
            divisor -= 1
        if j == size - 1 or m[i,j+1] > m[i,j]:
            divisor -= 1
        if i == size - 1 or m[i+1,j] > m[i,j]:
            divisor -= 1

        # if (i == size - 1 and j == size - 1) or m[i+1,j+1] > m[i,j]:
        #     divisor -= 1
        # if (i == 0 and j == 0) or m[i-1,j-1] > m[i,j]:
        #     divisor -= 1
        # if (i == 0 and j == size - 1) or m[i-1,j+1] > m[i,j]:
        #     divisor -= 1
        # if (j == 0 and i == size - 1) or m[i+1,j-1] > m[i,j]:
        #     divisor -= 1

        if divisor == 0:
            return

        loss_from_center = 0

        if i > 0 and m[i-1,j] < m[i,j]:
            exchange = (m[i,j] - m[i-1, j]) / divisor
            m[i-1,j] += exchange
            loss_from_center += exchange


        if j > 0 and m[i,j-1] < m[i,j]:
            exchange = (m[i,j] - m[i, j-1]) / divisor
            m[i,j-1] += exchange
            loss_from_center += exchange


        if j < size - 1 and m[i,j+1] < m[i,j]:
            exchange = (m[i,j] - m[i, j+1]) / divisor
            m[i,j+1] += exchange
            loss_from_center += exchange


        if i < size - 1 and m[i+1,j] < m[i,j]:
            exchange = (m[i,j] - m[i+1, j]) / divisor
            m[i+1,j] += exchange
            loss_from_center += exchange

        # if j < size - 1 and i < size - 1 and m[i+1,j+1] < m[i,j]:
        #     m[i+1,j+1] += m[i,j] / divisor
        # if i > 0 and j > 0 and m[i-1,j-1] < m[i,j]:
        #     m[i-1,j-1] += m[i,j] / divisor
        # if i > 0 and j < size - 1 and m[i-1,j+1] < m[i,j]:
        #     m[i-1,j+1] += m[i,j] / divisor
        # if j > 0 and i < size - 1 and m[i+1,j-1] < m[i,j]:
        #     m[i+1,j-1] += m[i,j] / divisor


        m[i,j] -= loss_from_center
    else:
        m[i,j] = .1


In [3]:
size = 16
m = np.zeros((size, size))
m[int(size / 2), int(size / 2)] = 10

# 16x16 threads per block.
bs = 16
# Number of blocks in the grid.
bpg = math.ceil(size / bs)
# We prepare the GPU function.
f = calculate_drift[(bpg, bpg), (bs, bs)]

In [4]:
m_gpu = cuda.to_device(m)

In [5]:
%%timeit
f(m)

TypeError: not enough arguments for format string

In [ ]:
m = m_gpu.copy_to_host()

In [ ]:
%reload_ext autoreload
%autoreload

import numpy as np
import creature_spawning.spawn_creature as sc
import visualization.creature_rendering as creature_render
import visualization.array_rendering as ar
import simulation.gas_drift as gd
import simulation.plant_simulation as ps

world_params_test_2 = {
    'world_size': 3,
    'carbon_dioxide_map': [[0, 0 , 0],
                                      [0, 5, 0],
                                      [0, 0, 0]]}
ar.render_array(world_params_test_2['carbon_dioxide_map'])

for i in range(10):
    gd.move_gases(world_params_test_2)
    ar.render_array(world_params_test_2['carbon_dioxide_map'])